# EDGES STATISTICAL SIGNIFICANCE TEST


In this notebook, I will determine whether including edges in the graph structure is statistically significant. I hope it is.

First, we must generate our data:

## Generating Data

We will create and train a total of 40 models -- 20 of which with a total of 180 edges (ie 5 edegs connecting each node), 20 of which with 36 edges (no edges connecting each node, only a self-loop). For each epoch in 100 epochs, we will record the training loss. Then, we will take the minimum??? of the list of losses for each model. We will create a list of the minimum epoch loss for every model to use in our t-test.

As a note, all configs will have the same hyperparameters, as determined in chem_talk_figures.

In [66]:
# ipython extension to autoreload imported modules so that any changes will be up to date before running code in this nb
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import ml_collections

from utils.jraph_training import train_and_evaluate_with_data, create_dataset, evaluate_model, rollout
# from utils.jraph_models import MLPGraphNetwork
from utils.jraph_data import print_graph_fts
from utils.jraph_vis import plot_predictions
import ml_collections
import optuna 
from flax import linen as nn
from functools import partial
from datetime import datetime
import os 
import tempfile
import matplotlib.pyplot as plt 
import numpy as np 

import jax
from utils.jraph_training import create_dataset, create_model, create_optimizer
from clu import checkpoint
from clu import parameter_overview
from flax.training import train_state

In [68]:
def create_config(seed, connected=None):
    config = ml_collections.ConfigDict()

    # Data params. 
    config.n_samples=5000
    config.input_steps=1
    config.output_delay=0 # predict 0 hours into the future
    config.output_steps=4
    config.timestep_duration=3
    config.sample_buffer=-1 * (config.input_steps + config.output_delay + config.output_steps - 1) # negative buffer so that our sample input are continuous (i.e. the first sample would overlap a bit with consecutive samples) 
    config.time_resolution=120
    config.init_buffer_samples=0
    config.train_pct=0.7
    config.val_pct=0.2
    config.test_pct=0.1
    config.K=36
    config.F=8
    config.c=10
    config.b=10
    config.h=1
    config.seed=seed+64
    config.normalize=True
    config.fully_connected_edges=connected

    # Optimizer.
    config.optimizer = 'sgd'
    config.learning_rate = 0.00045346796177033903
    config.momentum = 0.8712873602503628

    # Training hyperparameters.
    # config.batch_size = 3
    config.epochs = 50
    config.log_every_epochs = 1
    config.eval_every_epochs = 1
    config.checkpoint_every_epochs = 10
    config.max_checkpts_to_keep = None # None means keep all checkpoints

    # GNN hyperparameters.
    config.model = 'MLPGraphNetwork'
    config.n_blocks = 1
    config.activation = 'relu'
    config.dropout_rate = 0.013287043114620523
    config.skip_connections = False # This was throwing a broadcast error in add_graphs_tuples_nodes when this was set to True
    config.layer_norm = False # TODO perhaps we want to turn on later
    config.edge_features = (8, 8) # the last feature size will be the number of features that the graph predicts
    config.node_features = (32, 2)
    config.global_features = None
    config.share_params = False
    return config

In [69]:
num_trials = 10

In [70]:
connected_configs = []
solo_configs = []

for trial in range(num_trials):
    connected_config = create_config(trial, connected=False)
    solo_config = create_config(trial)

    connected_configs.append(connected_config)
    solo_configs.append(solo_config)

In [71]:
from utils.jraph_data import get_lorenz_graph_tuples

In [72]:
# set up logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [73]:
all_connected_datasets = []
all_solo_datasets = []
for connected_config, solo_config in zip(connected_configs, solo_configs):
    connected_datasets = get_lorenz_graph_tuples(
        n_samples=connected_config.n_samples,
        input_steps=connected_config.input_steps,
        output_delay=connected_config.output_delay,
        output_steps=connected_config.output_steps,
        timestep_duration=connected_config.timestep_duration,
        sample_buffer=connected_config.sample_buffer,
        time_resolution=connected_config.time_resolution,
        init_buffer_samples=connected_config.init_buffer_samples,
        train_pct=connected_config.train_pct,
        val_pct=connected_config.val_pct,
        test_pct=connected_config.test_pct,
        K=connected_config.K,
        F=connected_config.F,
        c=connected_config.c,
        b=connected_config.b,
        h=connected_config.h,
        seed=connected_config.seed,
        normalize=connected_config.normalize,
        fully_connected_edges=connected_config.fully_connected_edges)
    all_connected_datasets.append(connected_datasets)

    # this should generate a new data structure, but it will use the lorenz data from the connected dataset.
    solo_datasets = get_lorenz_graph_tuples(
        n_samples=solo_config.n_samples,
        input_steps=solo_config.input_steps,
        output_delay=solo_config.output_delay,
        output_steps=solo_config.output_steps,
        timestep_duration=solo_config.timestep_duration,
        sample_buffer=solo_config.sample_buffer,
        time_resolution=solo_config.time_resolution,
        init_buffer_samples=solo_config.init_buffer_samples,
        train_pct=solo_config.train_pct,
        val_pct=solo_config.val_pct,
        test_pct=solo_config.test_pct,
        K=solo_config.K,
        F=solo_config.F,
        c=solo_config.c,
        b=solo_config.b,
        h=solo_config.h,
        seed=solo_config.seed,
        normalize=solo_config.normalize,
        fully_connected_edges=solo_config.fully_connected_edges)
    all_solo_datasets.append(solo_datasets)

In [77]:
def create_workdirs(trials):
    connected_workdir_start = "tests/outputs/connected"
    solo_workdir_start = "tests/outputs/solo"
    connected_workdirs = []
    solo_workdirs = []
    for i in range(trials):
        connected_trial_workdir = os.path.join(connected_workdir_start, f"trial-{i}")
        solo_trial_workdir = os.path.join(solo_workdir_start, f"trial-{i}")
        connected_workdirs.append(connected_trial_workdir)
        solo_workdirs.append(solo_trial_workdir)
    return connected_workdirs, solo_workdirs

In [78]:
connected_workdirs, solo_workdirs = create_workdirs(num_trials)

trained_metrics = []
connected_eval_metrics = []
solo_eval_metrics = []

all_connected_epoch_losses = []
all_solo_epoch_losses = []

for trial in range(num_trials):
    _, _, connected_eval_metric, connected_epoch_losses = train_and_evaluate_with_data(
        config=connected_configs[trial], workdir=connected_workdirs[trial], datasets=all_connected_datasets[trial])
    
    _, _, solo_eval_metric, solo_epoch_losses = train_and_evaluate_with_data(
        config=solo_configs[trial], workdir=solo_workdirs[trial], datasets=all_solo_datasets[trial])
    
    connected_eval_metrics.append(connected_eval_metric)
    solo_eval_metrics.append(solo_eval_metric)
    
    all_connected_epoch_losses.append(connected_epoch_losses)
    all_solo_epoch_losses.append(solo_epoch_losses)

INFO:absl:Hyperparameters: {'F': 8, 'K': 36, 'activation': 'relu', 'b': 10, 'c': 10, 'checkpoint_every_epochs': 10, 'dropout_rate': 0.013287043114620523, 'edge_features': (8, 8), 'epochs': 50, 'eval_every_epochs': 1, 'fully_connected_edges': False, 'global_features': None, 'h': 1, 'init_buffer_samples': 0, 'input_steps': 1, 'layer_norm': False, 'learning_rate': 0.00045346796177033903, 'log_every_epochs': 1, 'max_checkpts_to_keep': None, 'model': 'MLPGraphNetwork', 'momentum': 0.8712873602503628, 'n_blocks': 1, 'n_samples': 5000, 'node_features': (32, 2), 'normalize': True, 'optimizer': 'sgd', 'output_delay': 0, 'output_steps': 4, 'sample_buffer': -4, 'seed': 64, 'share_params': False, 'skip_connections': False, 'test_pct': 0.1, 'time_resolution': 120, 'timestep_duration': 3, 'train_pct': 0.7, 'val_pct': 0.2}
INFO:absl:Initializing network.
INFO:absl:
+----------------------------------------+----------+------+----------+-------+
| Name                                   | Shape    | Siz

here we will take the average of the last 10 of 30 epochs.

In [85]:
print(len(all_connected_epoch_losses))

10


In [86]:
def average_last_epochs(average_after, epoch_losses):
    filtered_losses = epoch_losses[average_after:]
    avg_loss = np.mean(filtered_losses)
    return avg_loss

In [87]:
avg_connected_losses = []
avg_solo_losses = []
average_after_epoch = connected_configs[0].epochs - 10 # we want 10 epochs total, so we subtract to get where we should start averaging
for trial in range(num_trials):
    avg_connected_loss = average_last_epochs(average_after_epoch, all_connected_epoch_losses[trial])
    avg_solo_loss = average_last_epochs(average_after_epoch, all_solo_epoch_losses[trial])

    avg_connected_losses.append(avg_connected_loss)
    avg_solo_losses.append(avg_solo_loss)

In [88]:
print(avg_connected_losses)
print(avg_solo_losses)

[0.7615833, 0.7911881, 0.72747505, 0.7517646, 0.7260231, 0.76149464, 0.76684225, 0.76684225, 0.8784798, 0.7419151]
[1.2841399, 1.3033626, 1.3069439, 1.2640886, 1.2887369, 1.3235165, 1.3083115, 1.3083115, 1.4113057, 1.244005]


In [91]:
from scipy import stats

#print(np.var(avg_connected_losses), np.var(avg_solo_losses))
# variances are very similar

stats.ttest_ind(a=avg_connected_losses, b=avg_solo_losses, equal_var=True)


Ttest_indResult(statistic=-27.25188030052075, pvalue=4.3617636519204354e-16)